<a id="top"></a>
# Finding the Rotation Curve of an Asteroid or Comet with TESScut and Lightkurve
***
## Learning Goals

By the end of this tutorial, you will:
- Understand what a TESS cutout of a moving target is
- Use the `Tesscut` feature of `astroquery` to find a cutout of an asteroid or comet by name
- Plot light curves using lightkurve
- Find the rotation period of a bright asteroid using lightkurve

## Introduction

NASA's [Transiting Exoplanet Survey Satellite, or TESS](https://archive.stsci.edu/missions-and-data/tess), was designed to detect planets orbiting bright stars. There are a number of different kinds of [TESS data products](https://heasarc.gsfc.nasa.gov/docs/tess/data-products.html) available in the MAST archive, including:
- FFI (full-frame image) time series contain the entire image captured by the camera taken at different times; these are large views of the sky.
- Target Pixel Files (TPFs) are smaller subsets of full images focused on a single target. They are "postage-stamp" sized images separated by smaller time intervals than FFIs.
- Light Curve Files are derived from the TPFs using Simple Aperture Photometry (SAP). 

Though TESS was designed to study stars and the signatures of exoplanets orbiting them, inevitably other objects are observed within the FFIs. In particular, some asteroids and comets move through the observing sectors. [Pál et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJS..247...26P/abstract) noted:

>"While its primary mission avoids the vicinity of the ecliptic plane by approximately six degrees, the scale height of the solar system debris disk is large enough to place various small body populations in the field of view." 
    
Because small bodies like asteroids and comets are moving targets, they don't have fixed RA and Dec coordinates. The MAST archive's [TESScut](https://mast.stsci.edu/tesscut/) tool has built-in features to find all of the sectors that include observations of a moving target, so long as the object's name or ID can be understood by the [JPL Horizon ephemerades interface](https://ssd.jpl.nasa.gov/horizons/app.html#/). `TESScut` can then return a "cutout" of the full-frame images centered on the target as it moves through the observing sector. With this cutout, we can perform some simple aperture photometry on the target with `lightkurve`. 

In [Pál et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJS..247...26P/abstract), the bright asteroid 354 Eleonora (often just called "Eleonora") was found to have a rotation period of 4.27735 hours and had a mean magnitude of about 9.4 while observed by TESS. 

In this tutorial, we will use `TESScut` to download a cutout of the TESS observations of Eleonora and use `lightkurve` to create a light curve of Eleonora and find its rotation period. This simple method will work because Eleonora is a relatively bright asteroid. Generally, for fainter solar system objects, more complex photometry methods will be needed. We'll investigate why in one of the exercises. 

## Workflow
The workflow for this notebook consists of:
* [Imports](#Imports)
* [Searching for TESS Observations using astroquery](#Searching-for-TESS-Observations-Using-astroquery)
* [Investigating TESSCut Data](#Investigating-the-TESSCut-Data)
    * [File Structure and Auxillary Data](#File-Structure-and-Auxillary-Data)
    * [Plotting Individual Cutouts](#Plotting-Individual-Cutouts)
* [Plotting the Cutout in Lightkurve](#lightkurve)
    * [Loading a Moving Target](#Loading-a-Moving-Target)
    * [Create an Interactive Light Curve](#Create-an-Interactive-Light-Curve)
    * [Data Cleaning](#Data-Cleaning)
        * [Removing Blank Cadences](#Removing-Blank-Cadences)
        * [Removing Stellar Contamination](#Removing-Stellar-Contamination)
    * [Plotting a Combined Light Curve](#Plotting-a-Combined-Light-Curve) 
* [Finding the Rotation Period from Periodogram](#Period)
* [Exercises](#Exercises)

## Imports

- `astropy.io fits` for acccessing FITS files
- `astropy.table Table` for creating tidy tables of the data
- `numpy` to handle array functions
- `matplotlib.pyplot` for plotting data
- `astroquery.mast Tesscut` will create a cutout of the TESS data from the MAST Archive
- `lightkurve` contains many useful tools for plotting and analyzing light curves

In [ ]:
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from astroquery.mast import Tesscut
from astropy.visualization import time_support
import lightkurve as lk

%matplotlib inline

## Searching for TESS Observations Using `astroquery`

You can use the [TESScut webpage](https://mast.stsci.edu/tesscut/) to make a cutout of a TESS FFI time series; for a solar system object like an asteroid or comet, be sure to click on the "Moving Target" button at the top after the words "Create cutout based on."

To create a scriptable process for using TESScut, we can instead use the built-in [MAST functionality in astroquery to access TESScut](https://astroquery.readthedocs.io/en/latest/mast/mast.html#cutouts). Let's set our object name to "Eleonora."

In [ ]:
objname = 'Eleonora'

TESS observed distinct areas of the sky called Sectors. We can use `Tesscut.get_sectors` to see which (if any) sectors inlude observations of our object. Whenever using `Tesscut` with a moving target, we must always set `moving_target=True` (the default is `False`). 

In [ ]:
sector_table = Tesscut.get_sectors(objectname=objname, moving_target=True)
print(sector_table)

Next we'll use `Tess.get_cutouts` to, as the name implies, get the cutouts. Again, we must include the keyword `moving_target=True`.

We will also add the `size` keyword to choose how many pixels (in both width and height) we want our cutout to be. For this tutorial, we will choose a 10x10 cutout. This choice significantly affects the size of the `.fits` files and the time it will take to complete `get_cutouts`. 

You can specify e.g. `sector=6` when requesting a single cutout, but it's not necessary. Without specifying the sector, we will receive results for all sectors available. 

In [ ]:
hdulist = Tesscut.get_cutouts(objectname=objname, moving_target=True, size=10)
print('Length of hdulist =',len(hdulist))

## Investigating the TESScut Data
The result of `Tesscut.get_cutouts` is a list of `.fits` datasets; there is one element in the list per sector returned above. The contents of each element in `hdulist` are identical to the `.fits` files downloaded from the [TESSCut](https://mast.stsci.edu/tesscut/) website with the same selections (target name and size). 

### File Structure and Auxillary Data
Let's examine the *second* item in the list, because it will be the more interesting of the two, as we'll soon see.

In [ ]:
hdulist[1].info()

The result that we get is described in the [astrocut documentation on Path Focused Target Pixel Files](https://astrocut.readthedocs.io/en/latest/astrocut/file_formats.html#path-focused-target-pixel-files). Extension 0 is the header with metadata. Astronomers not used to working with moving targets may note that the `RA_OBJ` and `DEC_OBJ` keywords are not there!

Extension 1 is the data we most want in the form of a binary table; we'll look at that next.

But first, Extension 2, if present, is an image the size of the full-frame image (FFI) that the cutouts were drawn from. It's an array: the value is 1 if the pixel was used and 0 if the pixel was not. By displaying this array as an image, we can see the trajectory of our moving target through the TESS sector. 

In [ ]:
plt.imshow(hdulist[1][2].data)

It appears that for this sector Eleonora is partially outside of the sector during the course of the observations.

### Plotting Individual Cutouts

Returning to Extension 1, this is a binary table with 358 rows and 16 columns. We can print out the names of each column, along with information about the units and size of arrays (when applicable).

In [ ]:
hdulist[1][1].columns

To create our plot, we'll want to use the data in the 'FLUX' column. Note that it has a shape of (10,10), which is the size of the cutout we requested.

In [ ]:
# Create the subplot
fig,ax=plt.subplots(1,1)

# Use the flux data to create the image
imgplot=plt.imshow(hdulist[1][1].data[0]['FLUX'])
plt.colorbar(label='Flux (e-/s)')

# Add a title, units, and print RA/DEC
ax.set_title(f"{objname} {hdulist[1][1].data[0][0]:.1f} {hdulist[1][1].header['TUNIT1']}")
print('RA: ',hdulist[1][1].data[0][14])
print('Dec: ',hdulist[1][1].data[0][15])

Whoops! This doesn't look right. Recall that Eleonora is not always in the TESS field of view; we must be looking at data from before the asteroid enters the sector.

Let's view a later image instead, say at row (or "cadence") 100.

In [ ]:
# Let's use a parameter to set the row to 100.
# How does the listed time change as a result?
i=100

# As before, create the plot, add titles, print RA/Dec
fig,ax=plt.subplots(1,1)
imgplot=plt.imshow(hdulist[1][1].data[i]['FLUX'])
plt.colorbar()
ax.set_title(f"{objname} {hdulist[1][1].data[i][0]:.1f} {hdulist[1][1].header['TUNIT1']}")
print('RA: ',hdulist[1][1].data[i][14])
print('Dec: ',hdulist[1][1].data[i][15])

Now we see our target. Remember, we thought the asteroid would not be in the field of view the whole time; we now see that the target started out of the frame and moved in. Notice that roughly two days have elapsed and the target has moved to a different RA and Dec.

<a id="lightkurve"></a>
## Plotting the Cutout in `Lightkurve`

Though `lightkurve` has a `search_tesscut` function, at the time of this tutorial (September 2022) it does not work for moving targets.

### Loading a Moving Target
We'll need to save these .fits files locally to load them as Target Pixel Files into `lightkurve`. In a later exercise, you'll confirm that the files we write are the same as those from the [TESSCut](https://mast.stsci.edu/tesscut/) tool in a web browser. 

In [ ]:
nsectors=len(hdulist)
for i in range(nsectors): 
    hdulist[i].writeto(f"objname_{i}.fits", overwrite=True)

Now, we'll load each .fits file in as a `TessTargetPixelFile` object and collect all the sectors together into a `TargetPixelFileCollection` object; basically, a list of target pixel file (TPF) objects.

In [ ]:
for i in range(nsectors):
    tpf = lk.TessTargetPixelFile(f"objname_{i}.fits")
    if i==0: 
        tpfc=lk.TargetPixelFileCollection([tpf])
    else:
        tpfc.append(tpf)

### Create an Interactive Light Curve
For one TPF at a time, we can use the `interact` feature of `lightkurve` to easily view a light curve (on the left) as a function of time. On the right, we'll see the 10x10 image of Eleonora, overlaid with the aperture mask used for the aperture photometry that produced the light curve. You can view the [Interactively inspecting Target Pixel Files](https://docs.lightkurve.org/tutorials/1-getting-started/interactively-inspecting-data.html) tutorial to learn about more of the features of `interact` such as how to choose a custom aperture mask or change the screen stretch.

Let's view the second sector observed by TESS. Try clicking and dragging the "Cadence Number" bar to see how Eleonora moves through the field of view. Can you tell that it is rotating every few hours?

In [ ]:
tpfc[1].interact()

The light curve above confirms our suspicion that Eleonora was not viewable in the sector during the whole time of observation because it moved *into* the FFI during the observation. At the time of the writing of this tutorial, these cadences do not have a quality flag associated with them. 

We can check that all of the cadences loaded in this observation have a quality flag of zero:

In [ ]:
np.sum(tpfc[0].quality)

### Data Cleaning
#### Removing Blank Cadences

To remove the blank cadences, we will make the light curve for each TPF, collect them as a list in a `LightCurveCollection` object and then truncate the light curves.

In [ ]:
for i in range(nsectors):
    lc=tpfc[i].to_lightcurve()
    if i==0: 
        lcc=lk.LightCurveCollection([lc])
    else:
        lcc.append(lc)

Based on the `interact` screen above, the second lightkurve should be truncated for all rows before cadence number 59. In an exercise below, you can confirm for yourself that we should truncate the first lightkurve after cadence number 293. 



In [ ]:
lcc[0]=lcc[0].truncate(after=293,column='cadenceno')
lcc[1]=lcc[1].truncate(before=59,column='cadenceno')

#### Removing Stellar Contamination

The second light curve also shows two notable features around cadences 191-203 and 366-381. By examining the pixel data screen on the right, it's clear that Eleonora passed by two stars at those times. The flux from both stars is added into the aperture photometry and resulting light curve, so we should also remove those cadences from our light curve. If we don't, they may interfere with the periodogram. 

In this case, we can use `remove_outliers` to remove fluxes greater than 2 standard deviations from the median. In a later exercise, you can confirm that this does not modify the rest of the periodic fluctuation of the light curve.

In [ ]:
lcc[1]=lcc[1].remove_outliers(sigma_upper=2)

### Plotting a Combined Light Curve
Now we're ready to view the two light curves on one plot. 

In [ ]:
lcc.plot()

Admittedly, this view is not very helpful because of the large gap in time between observations of Eleonora. However, this time gap is important because it explains the difference in mean flux values between the two observations. Eleonora must have either been farther away from us during the first sector (and hence, dimmer) *or* possibly in a dimmer phase due to its position from us relative to the sun.

Still, having more data will help confirm the periodic nature of the flux variation, and hence, the period of the rotation of the asteroid. So, we should still stitch these together to create one light curve. In an exercse, you can investigate if this affects the rotation period.

In [ ]:
lc_stitched=lcc.stitch()
lc_stitched.plot()

<a id="period"></a>
## Finding the Rotation Period from Periodogram
Next, we'll create the periodogram of this stitched light curve. More information can be found in the lightkurve documentation: [What are Periodogram objects?](https://docs.lightkurve.org/tutorials/1-getting-started/what-are-periodogram-objects.html)

In [ ]:
pg=lc_stitched.to_periodogram()

In [ ]:
pg.plot()

There is clearly a frequency around 11 d<sup>-1</sup> found with maximum power. That corresponds to a period of:

In [ ]:
pg.period_at_max_power

Or, in hours:

In [ ]:
pg.period_at_max_power.to(u.hr)

We can use this period to fold the light curve. This plots the flux values as a function of phase, or progress along each iteration of the period. In the folded light curve, the horizontal axis spans a value of 0.089 days. Rather than be shown all from left to right in time, the data points go back to the left side of the graph every 0.089 days; that's why it's "folded."

In [ ]:
lc_stitched.fold(period=pg.period_at_max_power).scatter()

There appear to be two separate light curves: one with higher variation from the normalized flux, and one with lower variation. These correspond, respectively, to the first and second sectors, where the mean brightness of the asteroid was notably different. We can confirm this by examining each folded light curve separately.

In [ ]:
lcc[0].fold(period=pg.period_at_max_power).scatter()
lcc[1].fold(period=pg.period_at_max_power).scatter()

The folded light curve for the second sector looks a little suspicious because the same point in the phase (around 0.02) seems to show two different levels of flux. [Pál et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJS..247...26P/abstract), found a rotation period of 4.27735 hours; their period is almost a perfectly integer multiple of ours (times 2), which is known as a harmonic. Their period corresponds to second highest peak in our periodogram. We can view the difference this makes by looking at the folded light curve with our period times two:

In [ ]:
lcc[0].fold(period=2*pg.period_at_max_power).scatter()
lcc[1].fold(period=2*pg.period_at_max_power).scatter()

The accepted period (the second one we viewed) produced better folded light curves.

## Exercises

1. Confirm that the first light curve in our hdulist should be truncated after cadence number 293. 

In [ ]:
# Place for code for Exercise 1

2. Confirm that using a `remove_outlier` cutoff of `sigma_upper=2` did not affect the other cadences of the light curve for the second sector. At this point in the tutorial, the second sector's original light curve is still saved under the variable name `lc`; plot a graph of its time vs. flux columns and then overplot our sigma-clipped time and flux columns. Hint: You'll need to [load `time_support()` first](https://docs.astropy.org/en/stable/visualization/matplotlib_integration.html#plotting-times) in order to use the time column in matplotlib. 

In [ ]:
# Place for code for Exercise 2

3. Determine if the rotation period returned by `lightkurve` is different when using only one sector's light curve at a time. 

In [ ]:
# Place for code for Exercise 3

4. Use the interactive [TESSCut webpage](https://mast.stsci.edu/tesscut/) to download 10x10 cutouts for all sectors observing Eleonora. Be sure to select "Moving Target" at the top! Load the resulting fits files in and confirm that they are identical to the items in `hdulist` above. Hint: Astropy has a built-in method called [FITSDiff](https://docs.astropy.org/en/stable/io/fits/api/diff.html). 

In [ ]:
# Place for code for Exercise 4

5. Try recreating the procedure above to find the rotation period for a fainter (higher magnitude) small body from [Pál et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJS..247...26P/abstract) such as Hippodamia. What issues do you encounter in this case?

In [ ]:
# Place for code for Exercise 5

6. Use [astroquery's Minor Planet Center Queries (MPC)](https://astroquery.readthedocs.io/en/latest/mpc/mpc.html?highlight=mpc#ephemerides) `get_ephemeris` feature to investigate why the median flux for the first sector's observations is lower than for the second sector's observations. The times of each cadence are in the light curve's `time` column.

In [ ]:
# Place for code for Exercise 6

## Citations

If you use `astropy`, `lightkurve`, or `TESScut` for published research, please cite the
authors. Follow these links for more information about citations:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `lightkurve`](http://docs.lightkurve.org/about/citing.html)
* Cite [Brasseur et al. 2019](https://ui.adsabs.harvard.edu/abs/2019ascl.soft05007B/abstract) for TESScut

## About this Notebook

**Author:** Julia Kamenetzky<br>
**Keywords:** <br>
**Last Updated:**  Sep 2022<br>
**Next Review:** Feb 2023<br>
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 